# Cohere Document Search with LlamaIndex

This example shows how to use the Python [LlamaIndex](https://docs.llamaindex.ai/en/stable/) library to run a text-generation request against [Cohere's](https://cohere.com/) API, then augment that request using the text stored in a collection of local PDF documents.

**Requirements:**
- You will need an access key to Cohere's API key, which you can sign up for at (https://dashboard.cohere.com/welcome/login). A free trial account will suffice, but will be limited to a small number of requests.
- After obtaining this key, store it in plain text in your home in directory in the `~/.cohere.key` file.
- (Optional) Upload some pdf files into the `source_documents` subfolder under this notebook. We have already provided some sample pdfs, but feel free to replace these with your own.

## Set up the RAG workflow environment

In [71]:
from getpass import getpass
import os
from pathlib import Path

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.llms.cohere import Cohere
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

import chromadb
from chromadb import Collection

Set up some helper functions:

In [72]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

Make sure other necessary items are in place:

In [73]:
#try:
#    os.environ["COHERE_API_KEY"] = open(Path.home() / ".cohere.key", "r").read().strip()
#    os.environ["CO_API_KEY"] = open(Path.home() / ".cohere.key", "r").read().strip()
#except Exception:
#    print(f"ERROR: You must have a Cohere API key available in your home directory at ~/.cohere.key")

# Look for the source-materials folder and make sure there is at least 1 pdf file here
contains_pdf = False
directory_path = "./source_documents"
if not os.path.exists(directory_path):
    print(f"ERROR: The {directory_path} subfolder must exist under this notebook")
for filename in os.listdir(directory_path):
    contains_pdf = True if ".pdf" in filename else contains_pdf
if not contains_pdf:
    print(f"ERROR: The {directory_path} subfolder must contain at least one .pdf file")

## LLM

In [74]:
#llm = Cohere(api_key=os.environ["COHERE_API_KEY"])
llm = Ollama(model="llama3", request_timeout=30.0)

Without additional information, Cohere is unable to answer the question correctly. **Vector in fact awarded 109 AI scholarships in 2022.** Fortunately, we do have that information available in Vector's 2021-22 Annual Report, which is available in the `source_documents` folder. Let's see how we can use RAG to augment our question with a document search and get the correct answer.

## Ingestion: Load and store the documents from source-materials

Start by reading in all the PDF files from `source_documents`.

In [75]:
# Load the pdfs
pdf_folder_path = "./source_documents"
documents = SimpleDirectoryReader(input_files=[f"{pdf_folder_path}/Vanguard_ETF_Statutory_Prospectus_Single_VOE.pdf"]).load_data()
print(f"Number of source materials: {len(documents)}\n")

Number of source materials: 116



## Define an embeddings model

This embeddings model will convert the textual data from our PDF files into vector embeddings. These vector embeddings will later enable us to quickly find the chunk of text that most closely corresponds to our original query.

In [76]:
#embed_model = CohereEmbedding(
#    model_name="embed-english-v3.0",
#    input_type="search_query"
#)

embed_model = OllamaEmbedding(
    model_name="nomic-embed-text",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

In [89]:
db = chromadb.PersistentClient(path="./chroma_db")
db.delete_collection("test");
chroma_collection = db.get_or_create_collection("test")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, embed_model=embed_model)
query_engine = index.as_query_engine(
    llm=llm
    #node_postprocessors = [reranker]
)


## Storage: Store the documents in a vector database

## Retrieval: Now do a search to retrieve the chunk of document text that most closely matches our original query

## Setup retriever and reranker

In [90]:

search_query_retriever = index.as_retriever(llm=llm)
#reranker = CohereRerank()

## Query Response pipeline

In [91]:
def get_response_to_query(query):
    # search_query_retrieved_nodes = search_query_retriever.retrieve(query)
    # print(f"Search query retriever found {len(search_query_retrieved_nodes)} results")
    # print(f"First result example:\n{search_query_retrieved_nodes[0]}\n")
    query_engine = index.as_query_engine(
        llm=llm
        # streaming=True
        #node_postprocessors = [reranker]
    )
    result = query_engine.query(query)
    return result

## Get the Fund Name

In [92]:
fund_name = get_response_to_query("What is the name of the fund? Give only the name without additional comments. The name of the fund is: ")
print(f"Fund Name: {fund_name}")

Fund Name: Vanguard ETF.


## Get responses to key queries

In [86]:
queries = [
    "What is the investment strategy of the fund?",
    "What are the investment objectives of the fund?",
    "Who are the key people in the management team?",
    "What is the investment philosphy of the fund regarding ESG (Environmental, Social, and Governance)?",
    "What industries, markets, or types of securities is the fund want exposure to?",
    "What investment tools (derivatives, leverage, etc) does does the fund use to achieve their investment goals?"
]

In [93]:
responses = []

for query in queries:
    result = get_response_to_query(query)
    responses.append(result.response)

response_answer_pairs = zip(queries, responses)


In [94]:
response_answer_text = ""
for (query, response) in response_answer_pairs:
    response_answer_text = f"{response_answer_text}{query}\n{response}\n\n"

print(response_answer_text)

What is the investment strategy of the fund?
Under normal circumstances, each Fund will invest at least 80% of its assets in the stocks that make up its target index. A Fund may change its 80% policy only upon 60 days' notice to shareholders.

What are the investment objectives of the fund?
The Vanguard Value ETF seeks to track the performance of a benchmark index that measures the investment return of large-capitalization value stocks.

Who are the key people in the management team?
The key people in the management team include Walter Nejman, Portfolio Manager at Vanguard, who has co-managed the Fund since 2016; Gerard C. O'Reilly, Principal of Vanguard, who has managed the Fund since 1994 (co-managed since 2016); and Aaron Choi, CFA, Kenny Narzikul, CFA, and Nicholas Birkett, CFA, Portfolio Managers at Vanguard, who have co-managed the Fund since August 2023.

What is the investment philosphy of the fund regarding ESG (Environmental, Social, and Governance)?
The provided context does

## Chat Engine

In [99]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    llm=llm,
    chat_mode="context",
    memory=memory,
    system_prompt=(
        f"You are an expert Mutual Fund analyst for a bank, and you privide answers to your boss about whether the bank should purchase the fund named {fund_name}."
    ),
)

In [100]:
chat_response = chat_engine.chat("What is the level of risk for the fund?")
print(chat_response)

Based on the statutory prospectus, I would say that the level of risk for the Vanguard ETF (VOE) is moderate to high. The fund invests in mid-capitalization value stocks, which are known for their volatility and potential to trail returns from the overall stock market.

The prospectus highlights several risks associated with the fund:

1. Investment style risk: Mid-cap stocks tend to be more volatile than large-cap stocks, and they often perform differently.
2. Index replicating risk: The fund may not be able to replicate its target index exactly, which could result in differences between the fund's performance and the underlying market.

Additionally, as an exchange-traded fund (ETF), VOE is subject to additional risks, such as:

1. Market price vs. NAV: The market price of ETF Shares may differ significantly from their net asset value (NAV).
2. Trading halts: Trading in ETF Shares may be halted due to individual or marketwide trading halts.
3. Delisting: ETF Shares may be delisted wi

In [101]:
chat_engine2 = index.as_chat_engine(
    llm=Ollama(model="blah", request_timeout=30.0),
    chat_mode="context",
    memory=memory,
    chat_history = chat_engine.chat_history,
    system_prompt=(
        f"You are an expert Mutual Fund analyst for a bank, and you privide answers to your boss about whether the bank should purchase the fund named {fund_name}."
    ),    
)
chat_response = chat_engine2.chat("is it higher or lower than most funds?")
print(chat_response)

AttributeError: property 'chat_history' of 'ContextChatEngine' object has no setter